In [2]:
import random

import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from torchsummary import summary
from sklearn.metrics import confusion_matrix

from modules import EarlyStopping, TestResult

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
input_size = 224
size = (input_size, input_size)
normalize_mean = (0.4914, 0.4822, 0.4465)
normalize_std = (0.2470, 0.2435, 0.2616)
batch_size = 16
num_workers = 8

transform_train = transforms.Compose([
    transforms.Resize(size=size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean, normalize_std)  
])
transform_validation = transforms.Compose([
    transforms.Resize(size=size),
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean, normalize_std)
])
transform_test = transforms.Compose([
    transforms.Resize(size=size),
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean, normalize_std)
])

train_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/train/train', 
    transform=transform_train
)
valid_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/train/valid', 
    transform=transform_validation
)
test_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/test', 
    transform=transform_test
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=num_workers,
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=num_workers,
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
)

In [4]:
train_loader.dataset

Dataset ImageFolder
    Number of datapoints: 2720
    Root location: /home/opticho/source/SimCLR/datasets/dataset2(3)/train/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))
           )

In [5]:
train_loader.sampler.num_samples

2720

In [6]:
total = 0
for x, y in train_loader:
    total += len(y)
print(total)

2720


In [7]:
class Model(torch.nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv2d = torch.nn.Conv2d(3, 3, 3, padding=1)
        self.net = EfficientNet.from_pretrained(
            'efficientnet-b0', 
            num_classes=3, 
            include_top = False)
        self.seq = torch.nn.Sequential(
            nn.BatchNorm1d(1280),
            nn.Dropout(0.5),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 3)
        )

    def forward(self, x):
        x = self.conv2d(x)
        x = self.net(x)
        x = x.view(-1, 1280)
        x = self.seq(x)
        
        return x

In [12]:
encoder = EfficientNet.from_pretrained('efficientnet-b0')
encoder

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [8]:
# net = Model()
net = EfficientNet.from_pretrained(
            'efficientnet-b0', 
            num_classes=3, 
            include_top = True)
net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

weights = [52, 18, 30]
class_weights = torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)

summary(net, (3, input_size, input_size))

Loaded pretrained weights for efficientnet-b0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [11]:
net._fc.in_features

1280

In [5]:
epochs = 100
patience = 5
save_path='./save'
early_stopping = EarlyStopping(
    patience=patience, 
    verbose=True, 
    save_path=save_path)

In [6]:
best_accuracy = 0.0
for epoch in range(epochs):
    train_accuracy, valid_accuracy = 0.0, 0.0
    train_loss, valid_loss = 0.0, 0.0
    
    # train
    net.train()
    losses = []
    correct, total = 0, 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels) # loss: tensor(loss, device, grad_fn)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        predicted = outputs.argmax(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    train_accuracy = correct / total * 100
    train_loss = np.average(losses)

    # validation
    net.eval()
    losses = []
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            loss = criterion(outputs, labels)

            losses.append(loss.item())
            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    valid_accuracy = correct / total * 100
    valid_loss = np.average(losses)

    lr = optimizer.param_groups[0]["lr"]
    print(f'[Epoch {epoch+1} / {epochs}] lr: {lr}')
    scheduler.step()
    print(f'Train - Loss: {train_loss:.6f}\tAccuracy: {train_accuracy:.2f}%')
    print(f'Valid - Loss: {valid_loss:.6f}\tAccuracy: {valid_accuracy:.2f}%')

    early_stopping(valid_loss, valid_accuracy, epoch, net, save=False)
    if early_stopping.early_stop:
        break


[Epoch 1 / 100] lr: 0.001
Train - Loss: 0.415685	Accuracy: 79.85%
Valid - Loss: 0.788307	Accuracy: 65.93%
[Epoch 2 / 100] lr: 0.0009000000000000001
Train - Loss: 0.251952	Accuracy: 87.39%
Valid - Loss: 0.755145	Accuracy: 68.44%
[Epoch 3 / 100] lr: 0.0008100000000000001
Train - Loss: 0.140851	Accuracy: 93.75%
Valid - Loss: 1.240263	Accuracy: 64.74%
EarlyStopping counter: 1 out of 5
[Epoch 4 / 100] lr: 0.000729
Train - Loss: 0.079914	Accuracy: 96.51%
Valid - Loss: 0.955216	Accuracy: 70.22%
EarlyStopping counter: 2 out of 5
[Epoch 5 / 100] lr: 0.0006561000000000001
Train - Loss: 0.095090	Accuracy: 96.54%
Valid - Loss: 0.837107	Accuracy: 73.33%
EarlyStopping counter: 3 out of 5
[Epoch 6 / 100] lr: 0.00059049
Train - Loss: 0.060658	Accuracy: 97.61%
Valid - Loss: 0.824299	Accuracy: 74.67%
EarlyStopping counter: 4 out of 5
[Epoch 7 / 100] lr: 0.000531441
Train - Loss: 0.041417	Accuracy: 98.38%
Valid - Loss: 0.873282	Accuracy: 74.22%
EarlyStopping counter: 5 out of 5
Early stop.
Best validatio

In [7]:
net.eval()
pred, true, soft = [], [], []
test_accuracy, test_loss = 0.0, 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # for majority voting
        softmax_fn = nn.Softmax(dim=1)
        softmax = softmax_fn(outputs).cpu().detach().tolist()
        for i in range(len(softmax)):
            soft.append(softmax[i])   

        predicted = outputs.argmax(1)

        pred_batch = predicted.cpu().numpy()
        true_batch = labels.cpu().numpy()
        for i in range(len(pred_batch)):
            pred.append(pred_batch[i].item())
            true.append(true_batch[i].item())


In [8]:
t = TestResult(pred, true, soft)
t.print_result()

Accuracy: 77.71 %

Confusion Matrix:
 [[311   0  91]
 [  0  87  46]
 [ 19  17 205]]

   class	sensitivity	specificity	support

   covid	0.77		0.95		402
 healthy	0.65		0.97		133
  others	0.85		0.74		241

   macro	0.76		0.89		776
weighted	0.78		0.89		776

